# FEM 2131/2132
---

# 3-4.5.1 Fatigue check for structural elements
## Determination of the permissible stresses for fatigue

12-01-2021 Pedro Biel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Packages

In [ ]:
!pip install version_information
!pip install xlsxwriter

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import xlsxwriter

In [ ]:
sys.path.insert(0, 'drive/My Drive/Colab Notebooks/80954 SOPC/packages/')
from steelvalues import SteelValues
from sigma_permissible_fatigue import PermissibleSigma
from tau_permissible_fatigue import PermissibleTau
from check_stress import PermissibleStress
from exportexcel import ExportExcel

# RSA stresses

Stresses in the bars obtained with the structure calculation program Autodesk Robot Structural Analysis Professional (RSA).

In [ ]:
cwd = r'/content/drive/My Drive/Colab Notebooks/80954 SOPC/xlsx'
xlsx = 'RSA stresses.xlsx'

In [ ]:
df = pd.read_excel(cwd + '/' + xlsx)

Note on the sign of $\sigma$ values in RSA:

- compression $\to$ positive value
- tension $\to$ negative value

In order to unify the sign criterion with the generally accepted criterion, the values are multiplied by (-1), in this way:

- compression $\to$ negative value
- tension $\to$ positive

In [ ]:
for col in list(df.columns):
    if col not in ['bar', 'node', 'component_group', 'noth_effect']:
        df[col] *= (-1)

In [ ]:
df

,bar,node,component_group,noth_effect,sigma_x_max_[MPa],sigma_x_min_[MPa],sigma_y_max_[MPa],sigma_y_min_[MPa],tau_xy_max_[MPa],tau_xy_min_[MPa]
0,200,36,E8,K2,-116,-77,0,0,0.7,-0.09
1,80,56,E8,K2,-99,-55,0,0,-1.0,-0.40
2,206,188,E8,K3,-112,-71,0,0,3.0,1.70
3,206,188,E8,K3,116,73,0,0,-2.6,-1.40
4,89,189,E8,K3,-97,-45,0,0,1.9,0.70
5,89,189,E8,K3,102,48,0,0,-1.7,-0.40
6,204,215,E8,K1,-108,-70,0,0,1.2,0.90
7,204,215,E8,K1,114,73,0,0,-0.9,-0.70
8,81,216,E8,K1,-93,-44,0,0,1.1,0.60
9,81,216,E8,K1,99,47,0,0,-0.9,-0.30


Being `sigma_x_max_[MPa]`, `sigma_y_max_[MPa]` and `tau_xy_max_[MPa]` the extreme stresses having the higher absolute value.

# Structural steel

| Steel (ISO) | Steel (EN 1993) | $f_y$ [MPa] | $f_u$ [MPa] |
| :---: | :---: | :---: | :---: |
| Fe 360 | S 235 | 235 | 360 |
| Fe 430 | S 275 | 275 | 430 |
| Fe 510 | S 355 | 355 | 490 |

Being:
- Steel (ISO) : steel grade according to ISO.
- Steel (EN 1993) : steel grade according to EN 1993.
- $f_y$ [MPa] : elastic limit according to EN 1993.
- $f_u$ [MPa] : ultimate tensile strengh according to EN 1993.

Although the EN 1993 values are used as they are considered more up to date, the ISO nomenclature is also followed.

In [ ]:
steel_grade = input('Steel grade (Fe 360, Fe 430, Fe 510, S 235, S 275, S 355): ')
if steel_grade not in ['Fe 360', 'Fe 430', 'Fe 510', 'S 235', 'S 275', 'S 355']:
    print('WARNING: wrong steel grade!')

Steel grade (Fe 360, Fe 430, Fe 510, S 235, S 275, S 355): Fe 430


In [ ]:
cwd = r'/content/drive/My Drive/Colab Notebooks/80954 SOPC/SQL'
db = 'structural_steel.db'

In [ ]:
table = 'EN_1993_1_1'
conn = sqlite3.connect(cwd + '/' + db)
df_steel = pd.read_sql('SELECT * FROM ' + table + ';', conn)

In [ ]:
steel_values = SteelValues(df_steel, steel_grade)
sigma_E = steel_values.elastic_limit()
sigma_R = steel_values.ultimate_tensile_strength()

In [ ]:
print(f'Steel grade               : {steel_grade}')
print(f'Elastic limit             : {sigma_E} MPa')
print(f'Ultimate tensile strength : {sigma_R} MPa')

Steel grade               : Fe 430
Elastic limit             : 275.0 MPa
Ultimate tensile strength : 430.0 MPa


# 3-4.4 Ratio $\kappa$ between the extreme stresses

Where $\sigma_{max}$ and $\sigma_{min}$ are the values of these extreme stresses, $\sigma_{max}$ being the extreme stress having the
higher absolute value, the ratio $\kappa$ may be written :

$$\kappa = \frac{\sigma_{min}}{\sigma_{max}}$$

or in the case of shear:

$$\kappa = \frac{\tau_{min}}{\tau_{max}}$$

In [ ]:
k_sx = round(df['sigma_x_min_[MPa]'] / df['sigma_x_max_[MPa]'], 3)
k_sy = round(df['sigma_y_min_[MPa]'] / df['sigma_y_max_[MPa]'], 3)
k_txy = round(df['tau_xy_min_[MPa]'] / df['tau_xy_max_[MPa]'], 3)
df['k_sx'] = k_sx
df['k_sy'] = k_sy
df['k_txy'] = k_txy
df = df.fillna(0)
cols = [
    'bar', 'node', 'component_group', 'noth_effect', 'k_sx', 'k_sy', 'k_txy'
    ]
df[cols]

,bar,node,component_group,noth_effect,k_sx,k_sy,k_txy
0,200,36,E8,K2,0.664,0.0,-0.129
1,80,56,E8,K2,0.556,0.0,0.400
2,206,188,E8,K3,0.634,0.0,0.567
3,206,188,E8,K3,0.629,0.0,0.538
4,89,189,E8,K3,0.464,0.0,0.368
5,89,189,E8,K3,0.471,0.0,0.235
6,204,215,E8,K1,0.648,0.0,0.750
7,204,215,E8,K1,0.640,0.0,0.778
8,81,216,E8,K1,0.473,0.0,0.545
9,81,216,E8,K1,0.475,0.0,0.333


# 3-4.5.1.1 Tensile and comprenssive loads

## Basic stress $\sigma_W$

The number of cycles and stress spectrum has been taken into account in determining the group classification for each member. A value of $\sigma_W$ can therefore be selected from a knowledge of the component classification group and the material concerned.

In [ ]:
db = 'sigmaW.db'

In [ ]:
conn = sqlite3.connect(cwd + '/' + db)
df_sW = pd.read_sql(
    'SELECT * FROM ' + steel_grade.replace(' ', '') + ';', conn
    )

Values of the basic stress $\sigma_W [N/mm²]$ depending on the component group and case of notch effect:

In [ ]:
df_sW

,component_group,W0,W1,W2,K0,K1,K2,K3,K4
0,E1,249.1,211.7,174.7,361.9,323.1,271.4,193.9,116.3
1,E2,224.4,190.7,157.1,293.8,262.3,220.3,157.4,94.4
2,E3,202.2,171.8,141.5,238.4,212.9,178.8,127.7,76.6
3,E4,182.1,154.8,127.5,193.5,172.8,145.1,103.7,62.2
4,E5,164.1,139.5,114.9,157.1,140.3,117.8,84.2,50.5
5,E6,147.8,125.7,103.5,127.5,113.8,95.6,68.3,41.0
6,E7,133.2,113.2,93.2,103.5,92.4,77.6,55.4,33.3
7,E8,120.0,102.0,84.0,84.0,75.0,63.0,45.0,27.0


Basic stress $\sigma_W$ for component group and notch effect:

In [ ]:
df = df.join(
    df_sW.set_index('component_group').stack().rename('sigma_W_[MPa]'), 
    on=['component_group','noth_effect']
    )
cols = ['bar', 'node', 'component_group', 'noth_effect', 'sigma_W_[MPa]']
df[cols]

,bar,node,component_group,noth_effect,sigma_W_[MPa]
0,200,36,E8,K2,63.0
1,80,56,E8,K2,63.0
2,206,188,E8,K3,45.0
3,206,188,E8,K3,45.0
4,89,189,E8,K3,45.0
5,89,189,E8,K3,45.0
6,204,215,E8,K1,75.0
7,204,215,E8,K1,75.0
8,81,216,E8,K1,75.0
9,81,216,E8,K1,75.0


## Permissible stresses $\sigma_t$ and $\sigma_c$ for fatigue

$\kappa \leq 0$

* for tension: 

$$\sigma_t = \sigma_W \cdot \frac{5}{3 - 2 \cdot \kappa} \leq 0,66 \cdot \sigma_E \tag*{(1)}$$

* for compression: 

$$\sigma_c = \sigma_W \cdot \frac{2}{1 - \kappa} \tag*{(2)}$$

$\kappa > 0$

* for tension: 

$$\sigma_t = \frac{\sigma_O}{1 - \left(1 - \dfrac{\sigma_O}{\sigma_{+1}}\right) \cdot \kappa} \leq 0,66 \cdot \sigma_E \tag*{(3)}$$

* for compression:

$$\sigma_c = 1,2 \cdot \sigma_t \tag*{(4)}$$

Where 

$\sigma_0$ = tensile stress for $\kappa$ = 0 is given by the formula (1), that is:

$$\sigma_0 = 1,66 \cdot \sigma_W$$

$\sigma_{+1}$ = tensile stress for $\kappa$ = + 1, that is, the ultimate strength $\sigma_W$ divided by the coefficient of safety 1,33:

$$\sigma_{+1} = 0,75 \cdot \sigma_R$$


In [ ]:
sigma_W = df['sigma_W_[MPa]']
k_sx = df['k_sx']
k_sy = df['k_sy']

In [ ]:
permissible_stress = PermissibleSigma(df, sigma_E, sigma_R)
tension_stress_x = permissible_stress.tension_stress_x()
compression_stress_x = permissible_stress.compression_stress_x()
tension_stress_y = permissible_stress.tension_stress_y()
compression_stress_y = permissible_stress.compression_stress_y()
df['sigma_tx_[MPa]'] = round(tension_stress_x, 1)
df['sigma_cx_[MPa]'] = round(compression_stress_x, 1)
df['sigma_ty_[MPa]'] = round(tension_stress_y, 1)
df['sigma_cy_[MPa]'] = round(compression_stress_y, 1)

In [ ]:
cols = [
    'component_group', 'noth_effect',
    'sigma_tx_[MPa]', 'sigma_cx_[MPa]', 'sigma_ty_[MPa]', 'sigma_cy_[MPa]'
    ]
df[cols]

,component_group,noth_effect,sigma_tx_[MPa],sigma_cx_[MPa],sigma_ty_[MPa],sigma_cy_[MPa]
0,E8,K2,181.5,-217.8,105.0,-126.0
1,E8,K2,167.5,-201.0,105.0,-126.0
2,E8,K3,145.7,-174.8,75.0,-90.0
3,E8,K3,144.6,-173.5,75.0,-90.0
4,E8,K3,116.1,-139.3,75.0,-90.0
5,E8,K3,117.1,-140.5,75.0,-90.0
6,E8,K1,181.5,-217.8,125.0,-150.0
7,E8,K1,181.5,-217.8,125.0,-150.0
8,E8,K1,175.5,-210.5,125.0,-150.0
9,E8,K1,175.8,-210.9,125.0,-150.0


# 3-4.5.1.2 Shear loads

## Basic stress $\sigma_{W0}$

Basic stress $\sigma_{W0}$ for component group and notch effect $W0$:

In [ ]:
df['sigma_W0_[MPa]'] = df['component_group'].map(
    df_sW.set_index('component_group')['W0']
    )
df[['bar', 'node', 'component_group', 'sigma_W0_[MPa]']]

,bar,node,component_group,sigma_W0_[MPa]
0,200,36,E8,120.0
1,80,56,E8,120.0
2,206,188,E8,120.0
3,206,188,E8,120.0
4,89,189,E8,120.0
5,89,189,E8,120.0
6,204,215,E8,120.0
7,204,215,E8,120.0
8,81,216,E8,120.0
9,81,216,E8,120.0


## Permissible stress $\tau_a$ for fatigue

For each of the group from E1 to E8 the permissible fatigue stress in tension of the case $W_0$ divided by $\sqrt{3}$ is taken:

$$\tau_a = \frac{\sigma_t \text{ of case } W_0}{\sqrt{3}}$$

In [ ]:
permissible_stress = PermissibleTau(df, sigma_E, sigma_R)
shear_stress = permissible_stress.shear_stress()
df['tau_a_[MPa]'] = round(shear_stress, 1)

In [ ]:
cols = ['bar', 'node', 'component_group', 'noth_effect', 'tau_a_[MPa]']
df[cols]

,bar,node,component_group,noth_effect,tau_a_[MPa]
0,200,36,E8,K2,104.8
1,80,56,E8,K2,104.8
2,206,188,E8,K3,104.8
3,206,188,E8,K3,104.8
4,89,189,E8,K3,104.8
5,89,189,E8,K3,104.8
6,204,215,E8,K1,104.8
7,204,215,E8,K1,104.8
8,81,216,E8,K1,104.8
9,81,216,E8,K1,104.8


# 3-4.5.1.3 Combined loads in tension, compression and shear

In this case the permissible stresses for fatigue for each normal load in tension or compression $\sigma_{xa}$, $\sigma_{ya}$ and shear $\tau_{xya}$ are determined by assuming that each acts separately.

Then the following three conditions are checked:

$$\frac{\sigma_{x,max}}{\sigma_{xa}} \le 1 \quad \quad \frac{\sigma_{y,max}}{\sigma_{ya}} \le 1 \quad \quad \frac{\tau_{xy,max}}{\tau_{xya}} \le 1$$

In addition to checking for each loading assumed to act alone, it is recommended
that the following relationship be checked:

$$\left(\frac{\sigma_{x,max}}{\sigma_{xa}}\right)^2 + \left(\frac{\sigma_{y,max}}{\sigma_{ya}}\right)^2 - \frac{\sigma_{x,max} \cdot \sigma_{y,max}}{|\sigma_{xa}| \cdot |\sigma_{ya}|} + \left(\frac{\tau_{xy,max}}{\tau_{xya}}\right)^2 \le 1$$

As this inequality constitutes a severe requirement, values slightly higher that 1 are acceptable, but in this case it is necessary to check the relation:

$$\sqrt{\left(\frac{\sigma_{x,max}}{\sigma_{xa}}\right)^2 + \left(\frac{\sigma_{y,max}}{\sigma_{ya}}\right)^2 - \frac{\sigma_{x,max} \cdot \sigma_{y,max}}{|\sigma_{xa}| \cdot |\sigma_{ya}|} + \left(\frac{\tau_{xy,max}}{\tau_{xya}}\right)^2} \le 1,05$$

In [ ]:
stress = PermissibleStress(df)
permissible_stress_sigma_x = stress.get_permissible_stress_sx()
permissible_stress_sigma_y = stress.get_permissible_stress_sy()
permissible_stress_tau_xy = stress.get_permissible_stress_txy()
ratio_sigma_x = stress.get_ratio_sigma_x()
ratio_sigma_y = stress.get_ratio_sigma_y()
ratio_tau_xy = stress.get_ratio_tau_xy()
ratio_1 = stress.get_ratio_1()
ratio_2 = stress.get_ratio_2()
df['sigma_xa_[MPa]'] = round(permissible_stress_sigma_x, 2)
df['sigma_ya_[MPa]'] = round(permissible_stress_sigma_y, 2)
df['tau_a_[MPa]'] = round(permissible_stress_tau_xy, 2)
df['ratio_s_x'] = round(ratio_sigma_x, 2)
df['ratio_s_y'] = round(ratio_sigma_y, 2)
df['ratio_t_xy'] = round(ratio_tau_xy, 2)
df['ratio_1'] = round(ratio_1, 2)
df['ratio_2'] = round(ratio_2, 2)
df['Validate'] = np.where(
    (df['ratio_1'] <= 1.0) | (df['ratio_2'] <= 1.05),
    'yes', 'no'
    )

## Maximum stresses

In [ ]:
cols = [
    'bar', 'node', 'component_group', 'noth_effect',
    'sigma_x_max_[MPa]', 'sigma_y_max_[MPa]', 'tau_xy_max_[MPa]',
    'sigma_xa_[MPa]', 'sigma_ya_[MPa]', 'tau_a_[MPa]'
    ]
df[cols]

,bar,node,component_group,noth_effect,sigma_x_max_[MPa],sigma_y_max_[MPa],tau_xy_max_[MPa],sigma_xa_[MPa],sigma_ya_[MPa],tau_a_[MPa]
0,200,36,E8,K2,-116,0,0.7,-217.8,105.0,104.8
1,80,56,E8,K2,-99,0,-1.0,-201.0,105.0,104.8
2,206,188,E8,K3,-112,0,3.0,-174.8,75.0,104.8
3,206,188,E8,K3,116,0,-2.6,144.6,75.0,104.8
4,89,189,E8,K3,-97,0,1.9,-139.3,75.0,104.8
5,89,189,E8,K3,102,0,-1.7,117.1,75.0,104.8
6,204,215,E8,K1,-108,0,1.2,-217.8,125.0,104.8
7,204,215,E8,K1,114,0,-0.9,181.5,125.0,104.8
8,81,216,E8,K1,-93,0,1.1,-210.5,125.0,104.8
9,81,216,E8,K1,99,0,-0.9,175.8,125.0,104.8


## Ratios

In [ ]:
cols = [
    'bar', 'node', 'component_group', 'noth_effect',
    'ratio_s_x', 'ratio_s_y', 'ratio_t_xy', 'ratio_1', 'ratio_2', 'Validate'
    ]
df[cols]

,bar,node,component_group,noth_effect,ratio_s_x,ratio_s_y,ratio_t_xy,ratio_1,ratio_2,Validate
0,200,36,E8,K2,0.53,0.0,0.01,0.28,0.53,yes
1,80,56,E8,K2,0.49,0.0,0.01,0.24,0.49,yes
2,206,188,E8,K3,0.64,0.0,0.03,0.41,0.64,yes
3,206,188,E8,K3,0.80,0.0,0.02,0.64,0.80,yes
4,89,189,E8,K3,0.70,0.0,0.02,0.49,0.70,yes
5,89,189,E8,K3,0.87,0.0,0.02,0.76,0.87,yes
6,204,215,E8,K1,0.50,0.0,0.01,0.25,0.50,yes
7,204,215,E8,K1,0.63,0.0,0.01,0.39,0.63,yes
8,81,216,E8,K1,0.44,0.0,0.01,0.20,0.44,yes
9,81,216,E8,K1,0.56,0.0,0.01,0.32,0.56,yes


# Export to Excel

In [ ]:
export_excel = ExportExcel(df)
export_excel.export_excel()

# Version

In [ ]:
%reload_ext version_information
%version_information numpy, pandas, version_information, sqlite3, xlsxwriter

Software versions
Python 3.6.9 64bit [GCC 8.4.0]
IPython 5.5.0
OS Linux 4.19.112+ x86_64 with Ubuntu 18.04 bionic
numpy 1.19.5
pandas 1.1.5
version_information 1.0.3
sqlite3 The 'sqlite3' distribution was not found and is required by the application
xlsxwriter 1.3.7
Mon Jan 25 11:06:49 2021 UTC